In [6]:
import pandas as pd

In [7]:
dates = ['2021', '2022', '2023', '2024', '202501', '202502', '202503', '202504', '202505']
df_full = pd.DataFrame()

for date in dates:
        df  = (pd.read_csv(f'../SPX_new/DAT_ASCII_SPXUSD_M1_{date}.csv', sep=';', header=None, names=['ts', 'open', 'high', 'low', 'close', 'volume'])
            .assign(ts = lambda d: pd.to_datetime(
                        d['ts'].str.replace(' ',''), format='%Y%m%d%H%M%S', utc=False))
            .set_index('ts')
            .tz_localize('Etc/GMT+5'))        # fixed EST = UTC-05:00
    
        # Convert to actual New-York time (will carry DST correctly)
        df.index = df.index.tz_convert('America/New_York')
        df.index = df.index.tz_localize(None)  

        # Keep only regular-session minutes
        regular = df.between_time("09:30", "16:06", inclusive="left").copy()
        regular.drop(columns='volume', inplace=True)
        df_full = pd.concat([df_full, regular], ignore_index=False)

In [8]:
df_full

,open,high,low,close
ts,,,,
2021-01-04 09:30:00,3765.797,3767.397,3764.048,3764.542
2021-01-04 09:31:00,3764.597,3765.052,3763.348,3763.348
2021-01-04 09:32:00,3763.331,3763.554,3761.033,3761.033
2021-01-04 09:33:00,3761.340,3761.340,3758.236,3759.399
2021-01-04 09:34:00,3759.536,3759.952,3758.034,3758.348
...,...,...,...,...
2025-05-16 16:01:00,5950.464,5952.139,5950.464,5952.009
2025-05-16 16:02:00,5951.841,5953.129,5951.573,5952.089
2025-05-16 16:03:00,5951.848,5952.475,5951.593,5952.409


In [9]:
print(df_full.index.dayofweek.unique())  # Check that we have no weekends
print(df_full.isna().sum())  # Check that we have no NaNs

Index([0, 1, 2, 3, 4], dtype='int32', name='ts')
open     0
high     0
low      0
close    0
dtype: int64


In [10]:
df_full.to_csv('../SPX_new/SPX_2021_2025.csv', index=True)